# Imports

In [1]:
!pip install transformers

In [2]:
import pandas as pd
import string
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
import re
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
# import torch
# from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup, DataCollatorForLanguageModeling, DataCollatorWithPadding, Trainer, TrainingArguments, AutoConfig
from tqdm import tqdm, trange
# import torch.nn.functional as F
import csv

2023-03-07 12:49:20.049313: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-07 12:49:20.171905: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-07 12:49:20.171922: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-07 12:49:20.218137: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-07 12:49:20.819682: W tensorflow/stream_executor/platform/de

## Pre-processing

In [3]:
data = pd.read_csv("../raw_data/final2.csv")

In [4]:
data = data.drop(columns = ['Unnamed: 0', 'classification'])

In [5]:
data = data.dropna(subset=['text'])

In [6]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [7]:
'Words in vocabulary: ', tokenizer.vocab_size

('Words in vocabulary: ', 50257)

In [8]:
tokenizer.tokenize

<bound method PreTrainedTokenizer.tokenize of GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True)})>

In [9]:
# def tokeniz(sentence):
#     var = tokenizer.tokenize(sentence )
#     text_ids = tokenizer.encode(var, return_tensors = 'pt')
#     return text_ids

In [10]:
data['token'] = data['text'].apply(tokenizer.encode)

Token indices sequence length is longer than the specified maximum sequence length for this model (2717 > 1024). Running this sequence through the model will result in indexing errors


In [11]:
data['text'][4]

'"  In the midst of a sandy desert somewhere in Asia the eyes of travellers are refreshed by the sight of a high mountain covered with beautiful trees among which the glitter of foaming waterfalls may be seen in the sunlight. In that clear still air it is even possible to hear the song of the birds and smell of the flowers; but though the mountain is plainly inhabited--for here and there a white tent is visible--none of the kings or princes who pass it on the road to Babylon or Baalbec ever plunge into its forests--or if they do they never come back. Indeed so great is the terror caused by the evil reputation of the mountain that fathers on their death-beds pray their sons never to try to fathom its mysteries. But in spite of its ill-fame a certain number of young men every year announce their intention of visiting it and as we have said are never seen again.  * * * * *  Now there was once a powerful king who ruled over a country on the other side of the desert and when dying gave the 

In [12]:
data['token'][4]

[1,
 220,
 554,
 262,
 15925,
 286,
 257,
 44039,
 10326,
 7382,
 287,
 7229,
 262,
 2951,
 286,
 37217,
 389,
 47193,
 416,
 262,
 6504,
 286,
 257,
 1029,
 8598,
 5017,
 351,
 4950,
 7150,
 1871,
 543,
 262,
 31133,
 286,
 11511,
 3723,
 1660,
 23348,
 743,
 307,
 1775,
 287,
 262,
 19606,
 13,
 554,
 326,
 1598,
 991,
 1633,
 340,
 318,
 772,
 1744,
 284,
 3285,
 262,
 3496,
 286,
 262,
 10087,
 290,
 8508,
 286,
 262,
 12734,
 26,
 475,
 996,
 262,
 8598,
 318,
 30723,
 30671,
 438,
 1640,
 994,
 290,
 612,
 257,
 2330,
 11105,
 318,
 7424,
 438,
 23108,
 286,
 262,
 21557,
 393,
 42676,
 508,
 1208,
 340,
 319,
 262,
 2975,
 284,
 28028,
 393,
 39648,
 9423,
 1683,
 32669,
 656,
 663,
 17039,
 438,
 273,
 611,
 484,
 466,
 484,
 1239,
 1282,
 736,
 13,
 9676,
 523,
 1049,
 318,
 262,
 3246,
 4073,
 416,
 262,
 6181,
 8507,
 286,
 262,
 8598,
 326,
 17150,
 319,
 511,
 1918,
 12,
 3077,
 82,
 12472,
 511,
 11989,
 1239,
 284,
 1949,
 284,
 277,
 32910,
 663,
 26045,
 13,
 887,
 287